<a href="https://colab.research.google.com/github/hugoheh/Gran_Kalendar/blob/main/eurosat_sup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb
import wandb
wandb.init()

     |████████████████████████████████| 1.7 MB 6.9 MB/s 
     |████████████████████████████████| 142 kB 71.9 MB/s 
     |████████████████████████████████| 97 kB 9.1 MB/s 
     |████████████████████████████████| 180 kB 68.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=eba68acba7645e3d9ef09fc90b68d1abdc0e32d52eda720a6af82d8edf32fa11
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ef7094cbe96fb945f0989bc04c8d94c2ce60f711ed9b7d289c3353a81158310a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
gpu_info = !nvidia-smi -i 0
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
# Yi
import numpy as np
import zipfile
import torchvision

from google.colab import files

Sun Jan 16 20:33:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!wget https://madm.dfki.de/files/sentinel/EuroSAT.zip

with zipfile.ZipFile('EuroSAT.zip', 'r') as zip_ref:
    zip_ref.extractall('eurosat')

--2022-01-16 20:33:31--  https://madm.dfki.de/files/sentinel/EuroSAT.zip
Resolving madm.dfki.de (madm.dfki.de)... 131.246.195.183
Connecting to madm.dfki.de (madm.dfki.de)|131.246.195.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94280567 (90M) [application/zip]
Saving to: ‘EuroSAT.zip’

EuroSAT.zip         100%[===================>]  89.91M  19.4MB/s    in 5.6s    

2022-01-16 20:33:37 (15.9 MB/s) - ‘EuroSAT.zip’ saved [94280567/94280567]



In [5]:
!pip install split-folders

In [6]:
import splitfolders
splitfolders.ratio('eurosat/2750', output="eurosat_new", seed=42, ratio=(.9, 0,0.1)) # 24300 train, 2700 test

Copying files: 27000 files [00:03, 8071.62 files/s]


In [7]:
parser = argparse.ArgumentParser(description='Train Sup on EuroSAT')

parser.add_argument('-a', '--arch', default='resnet18')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.06, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=200, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[120, 160], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=512, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

# utils
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

parser.add_argument('--pretrained', default='', type=str, metavar='PATH', help='pretrained weights')


'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 100
args.cos = True
args.schedule = []  # cos in use
args.batch_size = 256
args.lr = 0.2
args.pretrained = ''

if args.results_dir == '':
    args.results_dir = '/content/drive/MyDrive/supervised_learning' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S-moco")

print(args)

Namespace(arch='resnet18', batch_size=256, cos=True, epochs=100, lr=0.2, pretrained='', results_dir='/content/drive/MyDrive/supervised_learning2022-01-16-20-33-58-moco', resume='', schedule=[], wd=0.0005)


In [8]:

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(56),    
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    #transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])

# data prepare
train_data = torchvision.datasets.ImageFolder(root='eurosat_new/train', transform=train_transform)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

test_data = torchvision.datasets.ImageFolder(root='eurosat_new/test', transform=test_transform)
test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(len(train_data),len(test_data))

24300 2700


In [9]:
class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=10, arch=None, bn_splits=1):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x

In [10]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)

    total_acc, total_loss, total_num, train_bar = 0.0, 0.0, 0, tqdm(data_loader)
    for img,target in train_bar:
        img, target = img.cuda(non_blocking=True), target.cuda(non_blocking=True)

        out = net(img)
        loss = criterion(out,target.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size

        score = torch.sigmoid(out).detach().cpu()
        pred_labels = torch.argmax(score,axis=1)
        #average_precision = accuracy_score(target.cpu(), torch.argmax(score,axis=1)) * 100.0
        average_precision = (pred_labels == target.cpu()).float().sum().item()

        total_acc += average_precision

        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}, Acc: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num, total_acc / total_num))

    return total_loss / total_num, total_acc / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [11]:
# test
def test(net, test_data_loader, epoch, args):
    net.eval()
    with torch.no_grad():
        test_bar = tqdm(test_data_loader)
        total_top1 = 0
        total_num = 0
        for img, target in test_bar:
            img, target = img.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = net(img)
            
            score = torch.sigmoid(out).detach().cpu()
            pred_labels = torch.argmax(score,axis=1)
            total_num += img.size(0)
            total_top1 += (pred_labels == target.cpu()).float().sum().item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}%'.format(epoch, args.epochs, total_top1 / total_num * 100))

    return total_top1 / total_num * 100

In [12]:
model = ModelBase(feature_dim=10, arch=args.arch).cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)


# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

# training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    if epoch%10 == 1:
      test_acc_1 = test(model, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

    if epoch%20==1:
      files.download(args.results_dir + '/model_last.pth')

    


Train Epoch: [1/100], lr: 0.199951, Loss: 2.0946, Acc: 0.3829: 100%|██████████| 94/94 [00:59<00:00,  1.58it/s]
Test Epoch: [1/100] Acc@1:38.85%: 100%|██████████| 11/11 [00:05<00:00,  2.07it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Train Epoch: [2/100], lr: 0.199803, Loss: 1.2986, Acc: 0.5564: 100%|██████████| 94/94 [01:01<00:00,  1.54it/s]
Train Epoch: [3/100], lr: 0.199556, Loss: 1.0738, Acc: 0.6195: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [4/100], lr: 0.199211, Loss: 0.9468, Acc: 0.6591: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [5/100], lr: 0.198769, Loss: 0.8415, Acc: 0.7015: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [6/100], lr: 0.198229, Loss: 0.7920, Acc: 0.7183: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [7/100], lr: 0.197592, Loss: 0.7286, Acc: 0.7441: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [8/100], lr: 0.196858, Loss: 0.6686, Acc: 0.7700: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [9/100], lr: 0.196029, Loss: 0.6271, Acc: 0.7811: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [10/100], lr: 0.195106, Loss: 0.5712, Acc: 0.8012: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Train Epoch: [62/100], lr: 0.063188, Loss: 0.1463, Acc: 0.9514: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [63/100], lr: 0.060285, Loss: 0.1378, Acc: 0.9543: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [64/100], lr: 0.057422, Loss: 0.1375, Acc: 0.9545: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [65/100], lr: 0.054601, Loss: 0.1397, Acc: 0.9537: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [66/100], lr: 0.051825, Loss: 0.1377, Acc: 0.9550: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [67/100], lr: 0.049096, Loss: 0.1388, Acc: 0.9533: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [68/100], lr: 0.046417, Loss: 0.1368, Acc: 0.9549: 100%|██████████| 94/94 [01:02<00:00,  1.52it/s]
Train Epoch: [69/100], lr: 0.043792, Loss: 0.1267, Acc: 0.9564: 100%|██████████| 94/94 [01:02<00:00,  1.51it/s]
Train Epoch: [70/100], lr: 0.041221, Loss: 0.1305, Acc: 0.9567: 100%|██████████| 94/94 [01:01<00:00,  1.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Train Epoch: [82/100], lr: 0.015567, Loss: 0.1106, Acc: 0.9627: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [83/100], lr: 0.013926, Loss: 0.1100, Acc: 0.9621: 100%|██████████| 94/94 [01:02<00:00,  1.52it/s]
Train Epoch: [84/100], lr: 0.012369, Loss: 0.1092, Acc: 0.9645: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [85/100], lr: 0.010899, Loss: 0.1099, Acc: 0.9645: 100%|██████████| 94/94 [01:02<00:00,  1.51it/s]
Train Epoch: [86/100], lr: 0.009517, Loss: 0.1098, Acc: 0.9622: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [87/100], lr: 0.008225, Loss: 0.1111, Acc: 0.9626: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [88/100], lr: 0.007022, Loss: 0.1119, Acc: 0.9630: 100%|██████████| 94/94 [01:01<00:00,  1.52it/s]
Train Epoch: [89/100], lr: 0.005912, Loss: 0.1023, Acc: 0.9659: 100%|██████████| 94/94 [01:01<00:00,  1.53it/s]
Train Epoch: [90/100], lr: 0.004894, Loss: 0.1113, Acc: 0.9628: 100%|██████████| 94/94 [01:01<00:00,  1.